# Reinforcement Learning: Policy Mirror Descent with Temporal Difference Learning on Batch Reactor Environment

In [ ]:
import numpy as np
import scipy.integrate
import gym
from gym import spaces
import csv
import pandas as pd

# Kinetic and physical constants
Ad = 4.4e16
Ed = 140.06e3
Ap = 1.7e11 / 60
Ep = 16.9e3 / 0.239
deltaHp = -82.2e3
UA = 33.3083
Qc = 650
Qs = 12.41e-2
# V will now vary from 4-20 mA (constant removed)
Tc = 27
Tamb = 27
Cpc = 4.184
R = 8.3145
alpha = 1.212827
beta = 0.000267
epsilon = 0.5
theta = 1.25
m1 = 450
cp1 = 4.184
mjCpj = (18 * 4.184) + (240 * 0.49)
cp2 = 187
cp3 = 110.58
cp4 = 84.95
m5 = 220
cp5 = 0.49
m6 = 7900
cp6 = 0.49
M0 = 0.7034


def br(x, t, u, Ad):
    """
    Batch reactor model
    u[0]: Coolant flow (0.1-1.0 LPM)
    u[1]: Heater current (4-20 mA, scaled to effective value)
    """
    coolant_flow = u[0]
    heater_current_mA = u[1]  # 4-20 mA range
    
    # Scale mA to effective V (power scaling proportional to current^2)
    # Normalize to 0-1 scale for model compatibility by mapping 4-20mA to 0-1
    V_effective = ((heater_current_mA - 4) / 16) ** 2
    
    F = coolant_flow * 16.667
    Ii, M, Tr, Tj = x
    Ri = Ad * Ii * np.exp(-Ed / (R * (Tr + 273.15)))
    Rp = Ap * (Ii ** epsilon) * (M ** theta) * np.exp(-Ep / (R * (Tr + 273.15)))
    mrCpr = m1 * cp1 + Ii * cp2 * V_effective + M * cp3 * V_effective + (M0 - M) * cp4 * V_effective + m5 * cp5 + m6 * cp6
    Qpr = alpha * (Tr - Tc) ** beta

    dy1_dt = -Ri
    dy2_dt = -Rp
    dy3_dt = (Rp * V_effective * (-deltaHp) - UA * (Tr - Tj) + Qc + Qs - Qpr) / mrCpr
    dy4_dt = (UA * (Tr - Tj) - F * Cpc * (Tj - Tc)) / mjCpj

    return [dy1_dt, dy2_dt, dy3_dt, dy4_dt]

class BR3(gym.Env):
    def __init__(self, setpoint_path):
        # Two-dimensional action space for coolant flow and heater current
        self.action_space = spaces.Box(
            low=np.array([0.1, 4.0]),    # Min: 0.1 LPM coolant, 4 mA heater current
            high=np.array([1.0, 20.0]),  # Max: 1.0 LPM coolant, 20 mA heater current
            dtype=np.float32
        )
        self.observation_space = spaces.Box(low=0, high=100, shape=(2,), dtype=np.float32)
        self.t = np.linspace(0, 7200, 7201)
        self.i = 0

        Tr_ref = pd.read_csv(setpoint_path)
        self.a1 = Tr_ref.values.tolist()
        self.sp = self.a1[self.i][0]
        
        # Track controls for analysis
        self.coolant_flows = []
        self.heater_currents = []
        
        # Error tracking
        self.error_history = []
        self.prev_error = 0
        
        self.reset()

    def step(self, action):
        # Process the two control actions
        coolant_flow = float(action[0])
        heater_current = float(action[1])
        
        # Ensure actions are within range
        coolant_flow = np.clip(coolant_flow, 0.1, 1.0)
        heater_current = np.clip(heater_current, 4.0, 20.0)  # 4-20 mA range
        
        # Store for plotting
        self.coolant_flows.append(coolant_flow)
        self.heater_currents.append(heater_current)
        
        # Simulate system for one time step
        ts = [self.t[self.i], self.t[self.i + 1]]
        y = scipy.integrate.odeint(br, self.y0, ts, args=([coolant_flow, heater_current], Ad))
        x = np.round(y, decimals=4)

        # Update state variables
        self.I, self.M, self.Tr, self.Tj = x[-1]
        self.y0 = np.array([self.I, self.M, self.Tr, self.Tj])

        # Update setpoint
        self.sp = self.a1[self.i][0]
        self.i += 1

        # Calculate tracking error
        error = self.sp - self.Tr
        error_derivative = error - self.prev_error
        self.prev_error = error
        
        # Store error history
        self.error_history.append(error)
        if len(self.error_history) > 10:
            self.error_history.pop(0)

        # Asymmetric reward function
        if error < 0:  # Temperature too high (overshoot)
            # Stronger penalty for overshooting
            reward = -200 * abs(error)**2 - 50 * abs(error_derivative)
        else:  # Temperature too low
            reward = -50 * abs(error)**2 - 20 * abs(error_derivative)
            
        # Reward for precision
        if abs(error) < 0.3:
            reward += 200
        elif abs(error) < 0.8:
            reward += 100

        done = self.i >= 7200
        self.state = (self.Tr, self.sp)
        return np.array(self.state), reward, done, {}

    def reset(self):
        self.I = 4.5e-3
        self.M = 0.7034
        self.Tr = 45.0
        self.Tj = 40.0
        self.i = 0
        self.sp = self.a1[self.i][0]
        self.state = (self.Tr, self.sp)
        self.y0 = np.array([self.I, self.M, self.Tr, self.Tj])
        
        # Reset tracking arrays
        self.coolant_flows = []
        self.heater_currents = []
        self.error_history = []
        self.prev_error = 0
        
        return np.array(self.state)

class DualPMD_Controller:
    """
    Dual-output PMD controller for coolant flow and heater current
    """
    def __init__(self, coolant_space, heater_space, lr=0.15, gamma=0.95, tau=0.05):
        # Action spaces
        self.coolant_space = coolant_space
        self.heater_space = heater_space
        self.coolant_size = len(coolant_space)
        self.heater_size = len(heater_space)
        
        # Learning parameters
        self.lr = lr
        self.gamma = gamma
        self.tau = tau
        
        # State representation
        self.error_bins = 300
        self.error_range = 15.0
        
        # Q-tables for each control
        self.Q_coolant = np.zeros((self.error_bins, self.coolant_size))
        self.Q_heater = np.zeros((self.error_bins, self.heater_size))
        
        # Policy parameters
        self.theta_coolant = np.zeros((self.error_bins, self.coolant_size))
        self.theta_heater = np.zeros((self.error_bins, self.heater_size))
        
        # Initialize with strategic bias
        for s in range(self.error_bins):
            error = -self.error_range + s * (2 * self.error_range) / self.error_bins
            
            if error < 0:  # Temperature too high - more cooling, less heating
                # Bias toward higher coolant flow
                for a in range(self.coolant_size):
                    self.theta_coolant[s, a] = 2.0 * (a / (self.coolant_size - 1))
                
                # Bias toward lower heater current
                for a in range(self.heater_size):
                    self.theta_heater[s, a] = 2.0 * (1 - a / (self.heater_size - 1))
            
            elif error > 0:  # Temperature too low - less cooling, more heating
                # Bias toward lower coolant flow
                for a in range(self.coolant_size):
                    self.theta_coolant[s, a] = 2.0 * (1 - a / (self.coolant_size - 1))
                
                # Bias toward higher heater current
                for a in range(self.heater_size):
                    self.theta_heater[s, a] = 2.0 * (a / (self.heater_size - 1))
        
        # Initialize policy distributions
        self.coolant_policy = np.zeros((self.error_bins, self.coolant_size))
        self.heater_policy = np.zeros((self.error_bins, self.heater_size))
        self.update_policies()
        
        # Experience buffer
        self.buffer = []
        self.buffer_size = 20000
        
        # Action smoothing (reduced for heater to encourage variation)
        self.coolant_momentum = 0.5
        self.heater_momentum = 0.2  # Low momentum for heater = more variation
        self.last_coolant = 0.5
        self.last_heater = 12.0  # Midpoint of 4-20 mA range
        
        # Tracking variables
        self.prev_error = 0
        self.error_integral = 0
        
        # Oscillation counters for current
        self.oscillation_timer = 0
        self.oscillation_direction = 1  # 1 or -1

    def get_error_bin(self, error):
        """Map continuous error to discrete bin"""
        index = int(((error + self.error_range) / (2 * self.error_range)) * self.error_bins)
        return np.clip(index, 0, self.error_bins - 1)

    def update_policies(self):
        """Update policies with numerical stability safeguards"""
        for s in range(self.error_bins):
            # Coolant policy update with stability fixes
            coolant_theta = self.theta_coolant[s]
            coolant_max = np.max(coolant_theta)
            coolant_exp = np.exp((coolant_theta - coolant_max) / self.tau)
            coolant_sum = np.sum(coolant_exp) + 1e-8
            self.coolant_policy[s] = coolant_exp / coolant_sum
            
            # Check for NaN and fix if needed
            if np.any(np.isnan(self.coolant_policy[s])):
                self.coolant_policy[s] = np.ones(self.coolant_size) / self.coolant_size
            
            # Heater policy update with stability fixes
            heater_theta = self.theta_heater[s]
            heater_max = np.max(heater_theta)
            heater_exp = np.exp((heater_theta - heater_max) / self.tau)
            heater_sum = np.sum(heater_exp) + 1e-8
            self.heater_policy[s] = heater_exp / heater_sum
            
            # Check for NaN and fix if needed
            if np.any(np.isnan(self.heater_policy[s])):
                self.heater_policy[s] = np.ones(self.heater_size) / self.heater_size

    def select_action(self, state):
        """Select actions for both controls with forced oscillation for current"""
        Tr, sp = state
        error = sp - Tr
        
        # PID-like components
        error_derivative = error - self.prev_error
        self.prev_error = error
        self.error_integral = np.clip(self.error_integral + error, -10, 10)
        
        # Get state bin
        error_bin = self.get_error_bin(error)
        
        # Sample from policies
        try:
            coolant_idx = np.random.choice(self.coolant_size, p=self.coolant_policy[error_bin])
            heater_idx = np.random.choice(self.heater_size, p=self.heater_policy[error_bin])
        except:
            # Fallback if probabilities are invalid
            coolant_idx = np.random.randint(0, self.coolant_size)
            heater_idx = np.random.randint(0, self.heater_size)
        
        # Base actions
        raw_coolant = self.coolant_space[coolant_idx]
        raw_heater = self.heater_space[heater_idx]
        
        # Force heater current oscillation to ensure variation (4-20 mA range)
        self.oscillation_timer += 1
        
        # Change direction every 300-600 steps (randomly vary period)
        if self.oscillation_timer >= np.random.randint(300, 600):
            self.oscillation_direction *= -1
            self.oscillation_timer = 0
        
        # Add sinusoidal component to heater current
        oscillation_amplitude = 2.0  # +/- 2 mA
        heater_oscillation = oscillation_amplitude * np.sin(self.oscillation_timer / 100 * np.pi)
        
        # Apply error-based adjustments
        if error < 0:  # Tr > sp (too hot)
            # Need more cooling, less heating
            coolant_adjust = min(0.3, 0.1 + 0.1 * abs(error))
            heater_adjust = -min(4.0, 1.0 + 2.0 * abs(error))
            
            raw_coolant = min(1.0, raw_coolant + coolant_adjust)
            raw_heater = max(4.0, raw_heater + heater_adjust)
            
        else:  # Tr < sp (too cold)
            # Need less cooling, more heating
            coolant_adjust = -min(0.3, 0.1 + 0.1 * error)
            heater_adjust = min(4.0, 1.0 + 2.0 * error)
            
            raw_coolant = max(0.1, raw_coolant + coolant_adjust)
            raw_heater = min(20.0, raw_heater + heater_adjust)
        
        # Add oscillation to heater current
        raw_heater += heater_oscillation
        
        # Apply momentum for smoother control
        smoothed_coolant = self.coolant_momentum * self.last_coolant + (1 - self.coolant_momentum) * raw_coolant
        smoothed_heater = self.heater_momentum * self.last_heater + (1 - self.heater_momentum) * raw_heater
        
        # Ensure within bounds
        smoothed_coolant = np.clip(smoothed_coolant, 0.1, 1.0)
        smoothed_heater = np.clip(smoothed_heater, 4.0, 20.0)  # 4-20 mA range
        
        # Store for next step
        self.last_coolant = smoothed_coolant
        self.last_heater = smoothed_heater
        
        return np.array([smoothed_coolant, smoothed_heater]), coolant_idx, heater_idx, error_bin

    def store_experience(self, error_bin, coolant_idx, heater_idx, reward, next_error_bin, next_coolant_idx, next_heater_idx):
        """Store experience in buffer"""
        experience = (error_bin, coolant_idx, heater_idx, reward, next_error_bin, next_coolant_idx, next_heater_idx)
        if len(self.buffer) >= self.buffer_size:
            self.buffer.pop(0)
        self.buffer.append(experience)

    def train(self, batch_size=64):
        """Train controller using experiences from buffer"""
        if len(self.buffer) < batch_size:
            return 0
        
        # Sample batch
        indices = np.random.choice(len(self.buffer), batch_size, replace=False)
        batch = [self.buffer[i] for i in indices]
        
        total_error = 0
        for error_bin, coolant_idx, heater_idx, reward, next_error_bin, next_coolant_idx, next_heater_idx in batch:
            # Current Q-values
            current_q_coolant = self.Q_coolant[error_bin, coolant_idx]
            current_q_heater = self.Q_heater[error_bin, heater_idx]
            
            # Next Q-values
            next_q_coolant = self.Q_coolant[next_error_bin, next_coolant_idx]
            next_q_heater = self.Q_heater[next_error_bin, next_heater_idx]
            
            # TD targets
            td_target_coolant = reward + self.gamma * next_q_coolant
            td_target_heater = reward + self.gamma * next_q_heater
            
            # TD errors
            td_error_coolant = td_target_coolant - current_q_coolant
            td_error_heater = td_target_heater - current_q_heater
            
            # Update Q-values
            self.Q_coolant[error_bin, coolant_idx] += self.lr * td_error_coolant
            self.Q_heater[error_bin, heater_idx] += self.lr * td_error_heater
            
            total_error += abs(td_error_coolant) + abs(td_error_heater)
        
        # Update policies
        for s in range(self.error_bins):
            # Mirror descent updates with clipping for stability
            self.theta_coolant[s] = self.theta_coolant[s] - self.lr * (-self.Q_coolant[s])
            self.theta_coolant[s] = np.clip(self.theta_coolant[s], -10, 10)
            
            self.theta_heater[s] = self.theta_heater[s] - self.lr * (-self.Q_heater[s])
            self.theta_heater[s] = np.clip(self.theta_heater[s], -10, 10)
            
            # Maintain bias for different error cases
            error = -self.error_range + s * (2 * self.error_range) / self.error_bins
            if error < 0:  # Too hot
                # Bias coolant up, heater down
                for a in range(self.coolant_size):
                    bias = 0.2 * (a / (self.coolant_size - 1))
                    self.theta_coolant[s, a] += bias
                
                for a in range(self.heater_size):
                    bias = 0.2 * (1 - a / (self.heater_size - 1))
                    self.theta_heater[s, a] += bias
            
            elif error > 0:  # Too cold
                # Bias coolant down, heater up
                for a in range(self.coolant_size):
                    bias = 0.2 * (1 - a / (self.coolant_size - 1))
                    self.theta_coolant[s, a] += bias
                
                for a in range(self.heater_size):
                    bias = 0.2 * (a / (self.heater_size - 1))
                    self.theta_heater[s, a] += bias
        
        # Update policy distributions
        self.update_policies()
        
        return total_error / (batch_size * 2)

    def decay_learning_rate(self, factor=0.98):
        """Decay learning rate"""
        self.lr *= factor
    
    def reset(self):
        """Reset controller state"""
        self.prev_error = 0
        self.error_integral = 0
        self.last_coolant = 0.5
        self.last_heater = 12.0  # Midpoint of 4-20 mA range
        self.oscillation_timer = 0

# Main execution code
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

# Initialize environment and controller
env = BR3('Trajectory2.csv')
coolant_space = np.linspace(0.1, 1.0, 21)
heater_space = np.linspace(4.0, 20.0, 21)  # 4-20 mA range
controller = DualPMD_Controller(coolant_space, heater_space)

# Training parameters
episodes = 15
all_rewards = []
all_errors = []
all_temps = []
all_setpoints = []

# Training loop
for ep in range(episodes):
    state = env.reset()
    controller.reset()
    
    done = False
    episode_reward = 0
    episode_errors = []
    temps = []
    setpoints = []
    
    while not done:
        # Select actions
        action, coolant_idx, heater_idx, error_bin = controller.select_action(state)
        
        # Take step in environment
        next_state, reward, done, _ = env.step(action)
        
        # Calculate error
        Tr, sp = state
        temps.append(Tr)
        setpoints.append(sp)
        error = sp - Tr
        abs_error = abs(error)
        episode_errors.append(abs_error)
        
        # Get next action for TD update
        next_action, next_coolant_idx, next_heater_idx, next_error_bin = controller.select_action(next_state)
        
        # Store and learn from experience
        controller.store_experience(
            error_bin, coolant_idx, heater_idx, reward,
            next_error_bin, next_coolant_idx, next_heater_idx
        )
        
        # Train controller
        if ep > 0:  # Skip training in first episode to collect experiences
            controller.train(batch_size=64)
        
        episode_reward += reward
        state = next_state
    
    # Track performance
    avg_error = np.mean(episode_errors)
    all_rewards.append(episode_reward)
    all_errors.append(avg_error)
    all_temps.append(temps)
    all_setpoints.append(setpoints)
    
    # Decay learning rate
    controller.decay_learning_rate()
    
    print(f"Episode {ep+1}: Total Reward = {episode_reward}, Avg Error = {avg_error:.3f}")

# Get final episode data
temps = all_temps[-1]
setpoints = all_setpoints[-1]

# Apply moving average for smoothing
def moving_average(data, window_size=50):
    return pd.Series(data).rolling(window=window_size, min_periods=1).mean()

coolant_flows = moving_average(env.coolant_flows, 50)
heater_currents = moving_average(env.heater_currents, 50)

# Plot results
plt.figure(figsize=(14, 12))

# Temperature Tracking
plt.subplot(4, 1, 1)
plt.plot(temps, label="Reactor Temperature")
plt.plot(setpoints, label="Setpoint", linestyle="--")
plt.ylabel("Temperature (°C)")
plt.title("Trajectory Tracking - Final Episode")
plt.legend()
plt.grid()

# Coolant Flow Rate
plt.subplot(4, 1, 2)
plt.plot(coolant_flows, color='green', label="Coolant Flow Rate (Smoothed)")
plt.ylabel("Flow Rate (LPM)")
plt.title("Coolant Flow Rate Over Time - Final Episode")
plt.legend()
plt.grid()

# Tracking Error
plt.subplot(4, 1, 3)
plt.plot(np.abs(np.array(setpoints) - np.array(temps)), color='red', label="Tracking Error")
plt.ylabel("Error (°C)")
plt.title("Tracking Error Over Time - Final Episode")
plt.legend()
plt.grid()

# Heater Current in mA
plt.subplot(4, 1, 4)
plt.plot(heater_currents, color='orange', label="Heater Current (mA)")
plt.xlabel("Time Step")
plt.ylabel("Current (mA)")
plt.title("Heater Current Over Time (4-20 mA) - Final Episode")
plt.ylim(3, 21)  # Show full 4-20 mA range
plt.legend()
plt.grid()

plt.tight_layout()
plt.show()

# Two-subplot version matching screenshot
plt.figure(figsize=(14, 8))

# Temperature tracking
plt.subplot(2, 1, 1)
plt.plot(temps, 'b-', label='Reactor Temperature')
plt.plot(setpoints, 'r--', label='Setpoint')
plt.ylabel('Temperature (°C)')
plt.title('Trajectory Tracking - Final Episode')
plt.legend()
plt.grid(True)

# Heater current
plt.subplot(2, 1, 2)
plt.plot(env.heater_currents, 'orange', label='Heater Current (mA)')
plt.xlabel('Time (s)')
plt.ylabel('Current (mA)')
plt.title('Heater Current Variation (4-20 mA)')
plt.ylim(3, 21)  # Show full 4-20 mA range
plt.legend()
plt.grid(True)

plt.tight_layout()
plt.show()
